# Tissue specific model [Competitive mechanism only]

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Brain_liver_specific_plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
M.reactions.Brain_biomass_maintenance.lower_bound=0

In [4]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = M.problem.Objective(M.reactions.Brain_biomass_maintenance.flux_expression+
                                              M.reactions.get_by_id('Brain_3HLYTCL').flux_expression+
                                             M.reactions.get_by_id('Brain_5HLTDL').flux_expression,
    direction='max')
M.objective = quadratic_objective

In [5]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

3.6218423879711157

In [6]:
M1=M.copy()

In [7]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = M1.problem.Objective(M1.reactions.Brain_biomass_maintenance.flux_expression+
                                              M1.reactions.get_by_id('Brain_3HLYTCL').flux_expression+
                                             M1.reactions.get_by_id('Brain_5HLTDL').flux_expression,
    direction='max')
M1.objective = quadratic_objective

In [9]:
## Brain Phe
M1.reactions.Brain_EX_phe_L_e__csf.lower_bound=0
M1.reactions.Brain_EX_phe_L_e__csf.upper_bound=0
M1.reactions.Brain_PHETA1.lower_bound=0
## Liver Phe_L_c
M1.reactions.Liver_PHETA1.lower_bound=0
M1.reactions.Liver_RE2269E.upper_bound=0
## Brain Tyr
M1.reactions.Brain_EX_tyr_L_e__csf.lower_bound=0
M1.reactions.Brain_EX_tyr_L_e__csf.upper_bound=0
M1.reactions.Brain_EX_34dhphe_e__csf.lower_bound=0
M1.reactions.Brain_EX_34dhphe_e__csf.upper_bound=0
M1.reactions.Brain_TYRTA.lower_bound=0
M1.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0
M1.reactions.Brain_TYRDOPO.upper_bound=0
## Brain Trp
M1.reactions.Brain_EX_trp_L_e__csf.lower_bound=0
M1.reactions.Brain_EX_trp_L_e__csf.upper_bound=0
## Reaction has upper_bound lower than 0, which will be changed to 0
M1.reactions.Liver_EX_ala_L_e.upper_bound=0
M1.reactions.Brain_EX_o2_e__csf.upper_bound=0
M1.reactions.Brain_EX_glc_D_e__csf.upper_bound=0
## Reaction has lower_bound higher than 0, which will be changed to 0
for rxn in M1.reactions:
    if M1.reactions.get_by_id(rxn.id).lower_bound>0:
        M1.reactions.get_by_id(rxn.id).lower_bound=0

In [10]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Brain_liver_specific_plusthreeCompetitive_map.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/Users/yanhualiu/Desktop/New_topic/model_building/model_building_exercise_by_yanhua/All code and Result (NEW) II (After deep discussion)/Using tissue specific model/utils/findBiomarkers.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [11]:
M1.solver='glpk'
sol = M1.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.4262626210295204

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.019361, …

In [12]:
## Model setting
M1.reactions.Liver_EX_tyr_L_e.lower_bound=-0.04
M1.reactions.Liver_EX_trp_L_e.lower_bound=-0.1
M1.reactions.Liver_EX_phe_L_e.lower_bound=-0.2
M1.reactions.Liver_EX_phe_L_e.upper_bound=-0.2
M1.reactions.Liver_biomass_reaction.lower_bound=0.05
M1.reactions.Liver_biomass_reaction.upper_bound=0.05
M1.reactions.Brain_biomass_maintenance.upper_bound=0.1
M1.reactions.Brain_5HLTDL.lower_bound=0.095
M1.reactions.Ti_To.upper_bound=0.195
M1.reactions.Liver_PHETA1.upper_bound=0
M1.reactions.Liver_TYRTA.lower_bound=0
M1.reactions.Liver_RE2269E.lower_bound=0
M1.reactions.Liver_HMR_7628.upper_bound=0
M1.reactions.Liver_r1455.upper_bound=0
M1.reactions.Liver_biomass_maintenance.upper_bound=0
M1.reactions.Brain_Phe_TH.upper_bound=0

In [13]:
M1.solver='glpk'
sol = M1.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.25175539999999996

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.019785, …

In [14]:
##PKU disease
M2=M1.copy()
M2.genes.get_by_id('5053').knock_out()
M2.reactions.Liver_PHETA1.upper_bound=0.1
M2.solver='glpk'
sol = M2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.1488514013504764

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': -0.01852, '…

# Other treatments

## tissue specific model with exercise (Other treatment)

In [15]:
model=M2.copy()

In [16]:
#Remembering to add other amino acid
model.reactions.Liver_EX_glc_D_e.lower_bound=-1.1##Do not need much glucsoe.
model.reactions.Liver_EX_his_L_e.lower_bound=-0.2
model.reactions.Liver_EX_ile_L_e.lower_bound=-0.2
model.reactions.Liver_EX_leu_L_e.lower_bound=-0.2
model.reactions.Liver_EX_lys_L_e.lower_bound=-0.2
model.reactions.Liver_EX_met_L_e.lower_bound=-0.2
model.reactions.Liver_EX_thr_L_e.lower_bound=-0.2
model.reactions.Liver_EX_trp_L_e.lower_bound=-0.1
model.reactions.Liver_EX_val_L_e.lower_bound=-0.2

In [17]:
model.reactions.Liver_biomass_reaction.upper_bound=0.5
sol = model.optimize()
sol.objective_value

0.197626889070242

In [18]:
fbasol = pfba(model)
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': 0.0, 'Liver…

## More carriers for model

In [19]:
model1=M2.copy()

In [20]:
model1.reactions.Ti_To.upper_bound=0.4
sol = model1.optimize()
sol.objective_value
fbasol = pfba(model1)
b = show_map(fbasol,map_loc)
b

0.2142269

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Liver_10FTHFtm': 0.0, 'Liver…